In [4]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:98% !important; }</style>"))

In [5]:
import uproot
import numpy as np
from tqdm import tqdm as tqdm
import matplotlib.pyplot as plt
import os
import optparse
import os.path as osp
import awkward as ak
import math
import torch_geometric
import torch
import sys
#import pandas as pd
from torch_geometric.data import Data
import glob

In [10]:
def analyze(filename,nfile,threshold, output_dir):



        filein = uproot.open(filename)["ticlDumper"]
        
        
        
        clusters = filein["clusters"].arrays()
        clusters = clusters[[x for x in ak.fields(clusters) if x not in ["run_","luminosityBlock_","event_"]]]
        tracksters = filein["trackstersCLUE3DHigh"].arrays()
        associations = filein["associations"].arrays()
        simtrackstersCP = filein["simtrackstersCP"].arrays()

        ## flatten to get better shapes
        simToReco_CP = ak.flatten(associations.tsCLUE3D_simToReco_CP)
        simToReco_CP_score = ak.flatten(associations.tsCLUE3D_simToReco_CP_score)

        ## choose the one with best score | best for closeby photons/electrons 
        #minsimscore_idxs = ak.argmin(simToReco_CP_score,axis=-1,keepdims=True )
        minsimscore_idxs = simToReco_CP_score == 1

        ## get corrsp trkster idxs
        matchtrk_idxs = simToReco_CP[minsimscore_idxs]


        #ratio_trksim = tracksters.raw_em_energy[matchtrk_idxs]/simtrackstersCP.raw_em_energy
        #ratio_trksim = ratio_trksim.to_numpy()

        #sel_clusters = clusters[tracksters.vertices_indexes[matchtrk_idxs]]
        ncalop = len(simtrackstersCP)



        listoftensors = []
        ##maskcalo = ak.argmin(sel_simmatch_scr[sel_simmatch_idxs == 0],axis=1,keepdims=True)
        for calop in range(ncalop):
            ntrks = len(tracksters.raw_energy[matchtrk_idxs][calop])
            #print("evt w ntrks ----------->",ntrks)
            if ntrks < 1:
                continue
                
            #if (ak.is_none(matchtrk_idxs[calop])):
            #    #print("was isnone skipped")
            #    continue
                
            ratio_trksim = tracksters.raw_em_energy[matchtrk_idxs][calop]/simtrackstersCP.raw_em_energy[calop]
            ratio_trksimtot = tracksters.raw_energy[matchtrk_idxs][calop]/simtrackstersCP.raw_energy[calop]
            
            print("n_trks:",ntrks)
            for ntrk in range(ntrks):
                #print("+++NTRK+++",ntrk)
                
                if (abs(tracksters.barycenter_z[matchtrk_idxs][calop][ntrk]) < 360)&(tracksters.raw_energy[matchtrk_idxs][calop][ntrk] > 4):
                    
                
                    gun_feats = np.stack((simtrackstersCP.boundaryEta[calop],
                                          simtrackstersCP.boundaryPhi[calop],
                                          simtrackstersCP.raw_em_energy[calop],
                                          simtrackstersCP.raw_energy[calop], 
                                          ratio_trksim[ntrk].reshape(1),
                                          ratio_trksimtot[ntrk].reshape(1))).to_numpy().T

                    #print("gun_feats",gun_feats.shape)
                    #print(gun_feats[:,3],gun_feats[:,4], gun_feats[:,5])

                    clus2d_feats = np.stack((ak.to_numpy(clusters.position_x[calop][ak.flatten(tracksters.vertices_indexes[matchtrk_idxs][calop][ntrk], axis=-1)]),
                                     ak.to_numpy(clusters.position_y[calop][ak.flatten(tracksters.vertices_indexes[matchtrk_idxs][calop][ntrk], axis=-1)]),
                                     ak.to_numpy(clusters.position_z[calop][ak.flatten(tracksters.vertices_indexes[matchtrk_idxs][calop][ntrk], axis=-1)]),
                                     ak.to_numpy(clusters.energy[calop][ak.flatten(tracksters.vertices_indexes[matchtrk_idxs][calop][ntrk], axis=-1)]),
                                     ak.to_numpy(clusters.cluster_time[calop][ak.flatten(tracksters.vertices_indexes[matchtrk_idxs][calop][ntrk], axis=-1)]),
                                     ak.to_numpy(clusters.cluster_layer_id[calop][ak.flatten(tracksters.vertices_indexes[matchtrk_idxs][calop][ntrk], axis=-1)]))).T


                    #print("clus2d_feats",clus2d_feats.shape)
                    #print(np.sum(clus2d_feats[:,3]))
                    #print( np.min(clus2d_feats[:,5]), np.max(clus2d_feats[:,5]))


                    clus3d_feats = np.stack((tracksters.barycenter_eta[matchtrk_idxs][calop][ntrk],
                                             tracksters.barycenter_phi[matchtrk_idxs][calop][ntrk],
                                             tracksters.raw_em_energy[matchtrk_idxs][calop][ntrk],
                                             tracksters.raw_energy[matchtrk_idxs][calop][ntrk],
                                             min(ak.to_numpy(clusters.cluster_layer_id[calop][ak.flatten(tracksters.vertices_indexes[matchtrk_idxs][calop][ntrk], axis=-1)])),
                                             max(ak.to_numpy(clusters.cluster_layer_id[calop][ak.flatten(tracksters.vertices_indexes[matchtrk_idxs][calop][ntrk], axis=-1)])))).T


                    #print("clus3d_feats",clus3d_feats.shape)
                    #print(clus3d_feats[3])
                    #print( clus3d_feats[4],clus3d_feats[5])

                    datum = Data(clus2d_feat = torch.tensor(clus2d_feats, dtype=torch.float32),
                                clus3d_feat = torch.tensor(clus3d_feats, dtype=torch.float32),
                                gun_feat = torch.tensor(gun_feats, dtype=torch.float32))

                    listoftensors.append(datum)
                    
                    

        processed_dir = output_dir #'/grid_mnt/data__data.polcms/cms/sghosh/NEWPID_DATA/PHO/' ## PHO
        import os
        import os.path as osp
        if not os.path.exists(processed_dir):
            os.makedirs(processed_dir)


        #torch.save(listoftensors, osp.join(processed_dir, 'data_{}_{}_{}_{}.pt'.format(nfile,evt,ngun,trkidx)))
        torch.save(listoftensors, osp.join(processed_dir, 'data_{}.pt'.format(nfile)))
        #print("skipped events:",nsskip)
        #print("average #tracksters:",np.mean(ntrkster))
        return 0

In [11]:
#raw_dir='/grid_mnt/data__data.polcms/cms/tarabini/GENPHOTESTPU2_noSmearing/step3_ticlv4_clue3d_CMSSW_12_4_0_pre2/'
#raw_dir="/grid_mnt/data__data.polcms/cms/cappati/240419_pions_step3/" ### PI
raw_dir="/grid_mnt/data__data.polcms/cms/cappati/240517_photonsD98_step3/"
output_dir = '/grid_mnt/data__data.polcms/cms/sghosh/NEWPID_TICLDUMPER_DATA/S2Rno_PUenrich_05082024/'
fnamelist = os.listdir(raw_dir)#+'STEP3_*.root'
print(fnamelist)

for idx, file in tqdm(enumerate(fnamelist)):
    print("processing file:",idx)
    if idx == 6:
        break
    #try:
        #print(raw_dir+file,idx)
    analyze(raw_dir+file,idx, 0.6, output_dir) ### PHO
        #analyze(raw_dir+file,idx, 0.99) ### PI
        #break
    #except:
    #    print("file is invalid, skipping....")

['step3_dumper_8.root', 'step3_dumper_1.root', 'step3_dumper_5.root', 'step3_dumper_14.root', 'step3_dumper_10.root', 'step3_dumper_2.root', 'step3_dumper_4.root', 'step3_dumper_18.root', 'step3_dumper_6.root', 'step3_dumper_7.root', 'step3_dumper_3.root', 'step3_dumper_17.root', 'step3_dumper_11.root', 'step3_dumper_15.root', 'step3_dumper_24.root', 'step3_dumper_12.root', 'step3_dumper_9.root', 'step3_dumper_28.root', 'step3_dumper_20.root', 'step3_dumper_25.root', 'step3_dumper_16.root', 'step3_dumper_34.root', 'step3_dumper_13.root', 'step3_dumper_29.root', 'step3_dumper_26.root', 'step3_dumper_38.root', 'step3_dumper_19.root', 'step3_dumper_27.root', 'step3_dumper_43.root', 'step3_dumper_37.root', 'step3_dumper_48.root', 'step3_dumper_21.root', 'step3_dumper_23.root', 'step3_dumper_30.root', 'step3_dumper_31.root', 'step3_dumper_45.root', 'step3_dumper_42.root', 'step3_dumper_47.root', 'step3_dumper_41.root', 'step3_dumper_32.root', 'step3_dumper_33.root', 'step3_dumper_36.root', 

0it [00:00, ?it/s]

processing file: 0
n_trks: 2809
n_trks: 2851
n_trks: 2893
n_trks: 2852
n_trks: 2815
n_trks: 3263
n_trks: 3182
n_trks: 2948
n_trks: 3792
n_trks: 3192
n_trks: 2038
n_trks: 3283
n_trks: 3544
n_trks: 3288
n_trks: 3064
n_trks: 2820
n_trks: 2850
n_trks: 3063
n_trks: 2818
n_trks: 3321
n_trks: 3035
n_trks: 2991
n_trks: 2708
n_trks: 3212
n_trks: 3069
n_trks: 2463
n_trks: 2246
n_trks: 3183
n_trks: 2874
n_trks: 3022
n_trks: 3428
n_trks: 2714
n_trks: 3785
n_trks: 2778
n_trks: 2418
n_trks: 2562
n_trks: 3844
n_trks: 3037
n_trks: 3349
n_trks: 3135
n_trks: 3061
n_trks: 3107
n_trks: 2842
n_trks: 2861
n_trks: 3815
n_trks: 2670
n_trks: 2764
n_trks: 3130
n_trks: 2940
n_trks: 2863
n_trks: 3609
n_trks: 2687
n_trks: 2743
n_trks: 2988
n_trks: 3043
n_trks: 2990
n_trks: 2819
n_trks: 3135
n_trks: 2927
n_trks: 2645
n_trks: 3479
n_trks: 2954
n_trks: 2717
n_trks: 2625
n_trks: 2933
n_trks: 2939
n_trks: 3506
n_trks: 2754
n_trks: 3019
n_trks: 3317
n_trks: 2967
n_trks: 2735
n_trks: 3196
n_trks: 3004
n_trks: 3070
n_trks

1it [50:43, 3043.37s/it]

processing file: 1
n_trks: 2888
n_trks: 2805
n_trks: 3201
n_trks: 3220
n_trks: 3434
n_trks: 2616
n_trks: 3303
n_trks: 2797
n_trks: 2668
n_trks: 3383
n_trks: 2640
n_trks: 3394
n_trks: 3250
n_trks: 3078
n_trks: 2744
n_trks: 3020
n_trks: 2932
n_trks: 3096
n_trks: 2890
n_trks: 2669
n_trks: 3356
n_trks: 2982
n_trks: 3152
n_trks: 2710
n_trks: 2959
n_trks: 3171
n_trks: 3143
n_trks: 2709
n_trks: 2380
n_trks: 3424
n_trks: 2987
n_trks: 3604
n_trks: 2811
n_trks: 2963
n_trks: 3306
n_trks: 2660
n_trks: 3129
n_trks: 2811
n_trks: 2459
n_trks: 2904
n_trks: 3020
n_trks: 3091
n_trks: 3218
n_trks: 3051
n_trks: 2734
n_trks: 2947
n_trks: 2701
n_trks: 2470
n_trks: 3236
n_trks: 3014
n_trks: 2903
n_trks: 2992
n_trks: 3211
n_trks: 3237
n_trks: 2972
n_trks: 3222
n_trks: 3216
n_trks: 3185
n_trks: 3300
n_trks: 2542
n_trks: 2883
n_trks: 3251
n_trks: 2837
n_trks: 3257
n_trks: 2764
n_trks: 2298
n_trks: 3665
n_trks: 2897
n_trks: 3594
n_trks: 2516
n_trks: 3483
n_trks: 3230
n_trks: 2651
n_trks: 2831
n_trks: 2888
n_trks

...

processing file: 4
n_trks: 2531
n_trks: 2711
n_trks: 3195
n_trks: 3064
n_trks: 3046
n_trks: 2709
n_trks: 3046
n_trks: 3050
n_trks: 3067
n_trks: 2900
n_trks: 2898
n_trks: 2804
n_trks: 3016
n_trks: 2495
n_trks: 2854
n_trks: 2942
n_trks: 3061
n_trks: 3012
n_trks: 2685
n_trks: 2937
n_trks: 3011
n_trks: 3051
n_trks: 3550
n_trks: 2265
n_trks: 2889
n_trks: 2671
n_trks: 3340
n_trks: 2957
n_trks: 3570
n_trks: 2773
n_trks: 2393
n_trks: 2794
n_trks: 3004
n_trks: 2811
n_trks: 3016
n_trks: 3214
n_trks: 3259
n_trks: 3136
n_trks: 3266
n_trks: 2985
n_trks: 2723
n_trks: 2676
n_trks: 3094
n_trks: 2804
n_trks: 2641
n_trks: 3342
n_trks: 2800
n_trks: 2826
n_trks: 3162
n_trks: 3035
n_trks: 2623
n_trks: 2889
n_trks: 3242
n_trks: 2967
n_trks: 3190
n_trks: 2744
n_trks: 3406
n_trks: 3090
n_trks: 3481
n_trks: 2848
n_trks: 3349
n_trks: 2472
n_trks: 3253
n_trks: 2562
n_trks: 3008
n_trks: 2710
n_trks: 3404
n_trks: 2870
n_trks: 2751
n_trks: 2796
n_trks: 2599
n_trks: 3520
n_trks: 3182
n_trks: 3035
n_trks: 3443
n_trks

5it [4:11:51, 3014.73s/it]

processing file: 5
n_trks: 2841
n_trks: 3078
n_trks: 3393
n_trks: 3147
n_trks: 2966
n_trks: 3440
n_trks: 3044
n_trks: 2762
n_trks: 2884
n_trks: 2332
n_trks: 2276
n_trks: 2625
n_trks: 3230
n_trks: 3249
n_trks: 2796
n_trks: 2662
n_trks: 3561
n_trks: 2931
n_trks: 2895
n_trks: 2856
n_trks: 2944
n_trks: 3431
n_trks: 3176
n_trks: 2881
n_trks: 2560
n_trks: 2415
n_trks: 3037
n_trks: 2755
n_trks: 3367
n_trks: 2980
n_trks: 3414
n_trks: 2763
n_trks: 3152
n_trks: 3003
n_trks: 3467
n_trks: 3207
n_trks: 2679
n_trks: 3362
n_trks: 3304
n_trks: 3474
n_trks: 3200
n_trks: 3208
n_trks: 2634
n_trks: 3226
n_trks: 2338
n_trks: 2718
n_trks: 3020
n_trks: 3050
n_trks: 3284
n_trks: 3245
n_trks: 3472
n_trks: 3034
n_trks: 3260
n_trks: 3226
n_trks: 3665
n_trks: 3092
n_trks: 2596
n_trks: 2738
n_trks: 3193
n_trks: 3419
n_trks: 3332
n_trks: 2847
n_trks: 3044
n_trks: 3135
n_trks: 3704
n_trks: 3921
n_trks: 3671
n_trks: 3126
n_trks: 2901
n_trks: 2455
n_trks: 3322
n_trks: 3363
n_trks: 3376
n_trks: 2589
n_trks: 3255
n_trks

6it [5:06:05, 3060.95s/it]

processing file: 6
